In [17]:
import os
import csv
from io import StringIO
from datetime import datetime
import numpy as np
import pandas as pd

# ============================
# FUNÇÕES AUXILIARES (ETL)
# ============================

def carregar_secoes_csv(conteudo_bytes: bytes) -> dict:
    """
    Lê um CSV em texto com seções iniciadas por '##NomeSecao'
    e devolve um dicionário {nome_secao: [linhas_csv]}
    """
    texto = conteudo_bytes.decode("utf-8-sig")
    linhas = texto.splitlines()

    secoes = {}
    secao_atual = None
    buffer = []

    for linha in linhas:
        linha = linha.strip()
        if linha.startswith("##"):
            if secao_atual and buffer:
                secoes[secao_atual] = buffer
                buffer = []
            secao_atual = linha.strip("#")
        elif linha:
            buffer.append(linha)

    if secao_atual and buffer:
        secoes[secao_atual] = buffer

    return secoes


def criar_dataframe(dados: list[str]) -> pd.DataFrame:
    """
    Recebe uma lista de linhas (strings) e converte em DataFrame.
    Supõe primeira linha como header.
    """
    if not dados:
        return pd.DataFrame()

    leitor = csv.reader(StringIO("\n".join(dados)))
    linhas = list(leitor)

    if len(linhas) < 2:
        return pd.DataFrame(columns=linhas[0]) if linhas else pd.DataFrame()

    return pd.DataFrame(linhas[1:], columns=linhas[0])


def tratar_decimal(df: pd.DataFrame, col: str) -> pd.Series:
    return pd.to_numeric(
        df[col].astype(str).str.replace(",", ".", regex=False),
        errors="coerce"
    )


def tratar_data_hora(df: pd.DataFrame, col_data: str) -> pd.DataFrame:
    df[col_data] = pd.to_datetime(df[col_data], errors="coerce")
    df["data"] = df[col_data].dt.date
    df["hora"] = df[col_data].dt.time
    return df


import os

def processar_arquivo(entrada):
    """
    Aceita:
      - bytes (conteúdo do arquivo)
      - str (caminho para o arquivo)
    Retorna: df_combustivel, df_despesas, df_servicos, df_rotas
    """
    # Se veio um caminho (str), lê o arquivo como bytes
    if isinstance(entrada, str):
        if not os.path.exists(entrada):
            raise FileNotFoundError(f"Arquivo não encontrado: {entrada}")
        with open(entrada, "rb") as f:
            conteudo_bytes = f.read()
    # Se já veio bytes, segue o jogo
    elif isinstance(entrada, (bytes, bytearray)):
        conteudo_bytes = bytes(entrada)
    else:
        raise TypeError(f"Entrada inválida: esperado str (caminho) ou bytes, veio {type(entrada)}")

    # daqui pra baixo, mantém seu fluxo original
    secoes = carregar_secoes_csv(conteudo_bytes)
    df_combustivel = criar_dataframe(secoes.get("Refuelling", []))
    df_despesas    = criar_dataframe(secoes.get("Expense", []))
    df_servicos    = criar_dataframe(secoes.get("Service", []))
    df_rotas       = criar_dataframe(secoes.get("Route", []))

    return df_combustivel, df_despesas, df_servicos, df_rotas


    # ============================
    # TRATAMENTO - DESPESAS
    # ============================
    if not df_despesas.empty:
        df_despesas["Odômetro (km)"] = tratar_decimal(df_despesas, "Odômetro (km)")
        df_despesas["Valor total"]   = tratar_decimal(df_despesas, "Valor total")
        df_despesas["Local da despesa"] = df_despesas["Local da despesa"].str.split(" / ").str[0]

        df_despesas = tratar_data_hora(df_despesas, "Data")

        df_despesas.rename(
            columns={
                "Odômetro (km)": "odometro",
                "Data": "data_hora",
                "Valor total": "custo",
                "Tipo de despesa": "tipo_despesa",
                "Local da despesa": "local",
                "Motorista": "motorista",
                "Forma de pagamento": "forma_pagamento",
                "Observação": "observacoes",
            },
            inplace=True,
        )

        df_despesas = df_despesas.astype(
            {
                "odometro": float,
                "custo": float,
                "tipo_despesa": str,
                "local": str,
                "motorista": str,
                "forma_pagamento": str,
                "observacoes": str,
            }
        )

    # ============================
    # TRATAMENTO - TRAJETOS (ROTAS)
    # ============================
    if not df_rotas.empty:
        df_rotas["Odômetro inicial"] = tratar_decimal(df_rotas, "Odômetro inicial")
        df_rotas["Odômetro final"]   = tratar_decimal(df_rotas, "Odômetro final")
        df_rotas["Odômetro (km)"]     = tratar_decimal(df_rotas, "Odômetro (km)")

        df_rotas["origem_tipo"] = df_rotas["Origem"].astype(str).str[-1]
        df_rotas["Origem"] = df_rotas["Origem"].str.split(" / ").str[0]
        df_rotas["destino_tipo"] = df_rotas["Destino"].astype(str).str[-1]
        df_rotas["Destino"] = df_rotas["Destino"].str.split(" / ").str[0]

        df_rotas["tipo_rota"] = df_rotas.apply(
            lambda row: "Urbano"
            if row["Origem"] == row["Destino"]
            else ("Urbano" if row["origem_tipo"] == "U" and row["destino_tipo"] == "U" else "Rodoviario"),
            axis=1,
        )

        df_rotas.rename(
            columns={
                "Data inicial": "data_inicio",
                "Data final": "data_fim",
                "Odômetro inicial": "odometro_inicial",
                "Odômetro final": "odometro_final",
                "Odômetro (km)": "odometro",
                "Valor km": "custo_km",
                "Total": "custo",
                "Origem": "origem",
                "Destino": "destino",
                "Motivo": "motivo",
                "Motorista": "motorista",
                "Observação": "observacoes",
            },
            inplace=True,
        )

        df_rotas["data_inicio"] = pd.to_datetime(df_rotas["data_inicio"], errors="coerce")
        df_rotas["data_fim"]    = pd.to_datetime(df_rotas["data_fim"], errors="coerce")
        df_rotas["tipo_rota"]   = df_rotas["tipo_rota"].astype("category")

    # ============================
    # TRATAMENTO - COMBUSTÍVEL
    # ============================
    if not df_combustivel.empty:
        df_combustivel.columns = [f"{col}_{i}" for i, col in enumerate(df_combustivel.columns)]
        df_combustivel.drop(df_combustivel.columns[7:17], axis=1, inplace=True)

        df_combustivel.rename(
            columns={
                "Odômetro (km)_0": "odometro",
                "Data_1": "data_hora",
                "Combustível_2": "combustivel",
                "Preço / L_3": "custo_por_litro",
                "Valor total_4": "custo",
                "Volume_5": "volume_abastecido",
                "Completou o tanque_6": "tanque_completo",
                "Média_17": "media",
                "Distância_18": "distancia_percorrida",
                "Posto de combustível_19": "posto",
                "Motorista_20": "motorista",
                "Motivo_21": "motivo",
                "Forma de pagamento_22": "forma_pagamento",
                "Observação_23": "observacoes",
            },
            inplace=True,
        )

        for c in ["odometro", "custo_por_litro", "custo", "volume_abastecido", "distancia_percorrida"]:
            if c in df_combustivel.columns:
                df_combustivel[c] = tratar_decimal(df_combustivel, c)

        if "media" in df_combustivel.columns:
            df_combustivel["media"] = pd.to_numeric(
                df_combustivel["media"]
                .astype(str)
                .str.replace(r"[^\d,.]", "", regex=True)
                .str.replace(",", ".", regex=False),
                errors="coerce",
            )

        if "data_hora" in df_combustivel.columns:
            df_combustivel = tratar_data_hora(df_combustivel, "data_hora")

        # =========================================
        # DISTÂNCIA CALCULADA ENTRE ABASTECIMENTOS
        # =========================================
        df_combustivel = df_combustivel.sort_values(
            by=["data_hora", "odometro"],
            ascending=[True, True],
            na_position="last"
        ).reset_index(drop=True)

        df_combustivel["odometro_proximo"] = df_combustivel["odometro"].shift(-1)
        df_combustivel["distancia_calculada"] = df_combustivel["odometro_proximo"] - df_combustivel["odometro"]
        df_combustivel.loc[df_combustivel["distancia_calculada"] < 0, "distancia_calculada"] = np.nan
        df_combustivel.drop(columns=["odometro_proximo"], inplace=True)

        # =========================================================
        # MÉDIA CALCULADA (km/L) - litros na linha atual, km nas anteriores
        # =========================================================
        df_combustivel = df_combustivel.sort_values(
            by=["data_hora"],
            ascending=True,
            na_position="last"
        ).reset_index(drop=True)

        flag_sim = (
            df_combustivel["tanque_completo"]
            .astype(str).str.strip().str.lower()
            .eq("sim")
        )

        segmento = flag_sim.cumsum()

        km_por_segmento = df_combustivel.groupby(segmento)["distancia_calculada"].sum(min_count=1)

        litros_nao_por_segmento = (
            df_combustivel.loc[~flag_sim]
            .groupby(segmento)["volume_abastecido"]
            .sum(min_count=1)
        )

        idx_sim = df_combustivel.index[flag_sim]
        seg_atual = segmento.loc[idx_sim]
        seg_base = seg_atual - 1

        km_base = km_por_segmento.reindex(seg_base).to_numpy()

        litros_base_nao = (
            litros_nao_por_segmento.reindex(seg_base)
            .fillna(0)
            .to_numpy()
        )

        litros_atual = df_combustivel.loc[idx_sim, "volume_abastecido"].to_numpy()
        litros_total = litros_base_nao + litros_atual

        media = km_base / litros_total

        df_combustivel["media_calculada"] = np.nan
        df_combustivel.loc[idx_sim, "media_calculada"] = media

        df_combustivel.loc[
            df_combustivel["media_calculada"].replace([np.inf, -np.inf], np.nan).isna(),
            "media_calculada"
        ] = np.nan
        df_combustivel.loc[df_combustivel["media_calculada"] < 0, "media_calculada"] = np.nan
        df_combustivel.loc[idx_sim[litros_total <= 0], "media_calculada"] = np.nan

        df_combustivel["media_calculada_preenchida"] = df_combustivel["media_calculada"].bfill()

        # =========================================================
        # MÉDIA ACUMULADA (km/L) - regra do "carro zero km tanque cheio"
        #
        # - Não calcula na 1ª linha
        # - Começa na 2ª linha: soma volume (linha0+linha1) e distância (linha0+linha1)
        # - Continua acumulando até o "primeiro registro mais recente" com tanque_completo == "Não"
        # =========================================================
        vol = pd.to_numeric(df_combustivel["volume_abastecido"], errors="coerce")
        dist = pd.to_numeric(df_combustivel["distancia_calculada"], errors="coerce")

        # acumulados "brutos" (cumsum ignora NaN por padrão; NaN vira "não soma")
        vol_cum = vol.cumsum()
        dist_cum = dist.cumsum()

        # não calcula na 1ª linha
        df_combustivel["volume_abastecido_acumulado"] = np.nan
        df_combustivel["distancia_calculada_acumulada"] = np.nan
        df_combustivel["media_acumulada"] = np.nan

        if len(df_combustivel) >= 2:
            df_combustivel.loc[1:, "volume_abastecido_acumulado"] = vol_cum.iloc[1:].to_numpy()
            df_combustivel.loc[1:, "distancia_calculada_acumulada"] = dist_cum.iloc[1:].to_numpy()

            denom = df_combustivel.loc[1:, "volume_abastecido_acumulado"].replace({0: np.nan})
            df_combustivel.loc[1:, "media_acumulada"] = (
                df_combustivel.loc[1:, "distancia_calculada_acumulada"] / denom
            )

        # Para de acumular no "primeiro registro mais recente" com tanque_completo == "Não"
        # (ou seja: remove a média acumulada na linha do "Não" e em todas as posteriores)
        flag_nao = (
            df_combustivel["tanque_completo"]
            .astype(str).str.strip().str.lower()
            .isin(["não", "nao"])
        )

        if flag_nao.any():
            stop_idx = flag_nao[flag_nao].index.max()  # mais recente (maior índice)
            df_combustivel.loc[
                stop_idx:,
                ["volume_abastecido_acumulado", "distancia_calculada_acumulada", "media_acumulada"]
            ] = np.nan


    # ============================
    # TRATAMENTO - SERVIÇOS (se existir)
    # ============================

    return df_combustivel, df_despesas, df_servicos, df_rotas


In [18]:
# ============================
# EXEMPLO DE USO (caminho)
# ============================
caminho = 'data/BD.csv'
df_combustivel, df_despesas, df_servicos, df_rotas = processar_arquivo(caminho)

In [19]:
df_combustivel.to_excel('data/combustivel.xlsx', index=False)

In [20]:
df_despesas.to_excel('data/despesas.xlsx', index=False)

In [21]:
df_rotas.to_excel('data/rotas.xlsx', index=False)

In [22]:
df_servicos.to_excel('data/servicos.xlsx', index=False)